In [1]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from lime import lime_image
from skimage.segmentation import slic
from scipy.stats import pearsonr

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cnn_model = load_model("/Users/samarpanpoudel/Desktop/model 1/saved_models/model1.h5")
resnet_model = load_model("/Users/samarpanpoudel/Desktop/Transfer Learning/saved_models/resnet1.h5")

In [3]:
def cnn_predict(images):
    return cnn_model.predict(images)

def resnet_predict(images):
    return resnet_model.predict(images)

In [4]:
explainer = lime_image.LimeImageExplainer()

def generate_lime_explanation(model_predict, img_array, segmentation):
    explanation = explainer.explain_instance(
        img_array,
        model_predict,
        top_labels=1,
        hide_color=0,
        num_samples=1000,
        segmentation_fn=lambda x: segmentation
    )
    return explanation

In [5]:
def get_importance_vector(explanation, label, num_superpixels):
    importance_dict = dict(explanation.local_exp[label])
    
    vector = np.zeros(num_superpixels)
    
    for superpixel_idx, weight in importance_dict.items():
        vector[superpixel_idx] = weight
        
    return vector

In [6]:
folder_path = "/Users/samarpanpoudel/Desktop/test"

image_files = sorted(os.listdir(folder_path))

correlations = []

for img_name in image_files:
    
    img_path = os.path.join(folder_path, img_name)
    
    img = image.load_img(img_path, target_size=(256,256))
    img_array = image.img_to_array(img)
    
    # Generate segmentation ONCE
    segmentation = slic(img_array, n_segments=50, compactness=10, sigma=1)
    num_superpixels = len(np.unique(segmentation))
    
    # LIME for CNN
    cnn_exp = generate_lime_explanation(cnn_predict, img_array, segmentation)
    cnn_label = cnn_exp.top_labels[0]
    cnn_vector = get_importance_vector(cnn_exp, cnn_label, num_superpixels)
    
    # LIME for ResNet
    resnet_exp = generate_lime_explanation(resnet_predict, img_array, segmentation)
    resnet_label = resnet_exp.top_labels[0]
    resnet_vector = get_importance_vector(resnet_exp, resnet_label, num_superpixels)
    
    # Compute correlation
    corr, _ = pearsonr(cnn_vector, resnet_vector)
    correlations.append(corr)
    
    print(f"{img_name} correlation: {corr}")

  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step


  1%|          | 10/1000 [00:00<00:15, 64.09it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


  3%|▎         | 30/1000 [00:00<00:09, 103.82it/s]

...

 99%|█████████▉| 990/1000 [00:41<00:00, 23.74it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 356ms/step


100%|██████████| 1000/1000 [00:41<00:00, 23.93it/s]

Lung MRI (483).png correlation: 0.19781741035533915



/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: i

In [7]:
correlations = np.array(correlations)

print("\nMean LIME Correlation:", np.mean(correlations))
print("Std:", np.std(correlations))


Mean LIME Correlation: 0.12912567677133316
Std: 0.14088375534364503


In [8]:
from scipy.stats import ttest_1samp
ttest_1samp(correlations, 0)

TtestResult(statistic=np.float64(3.03982116040705), pvalue=np.float64(0.011250189336343148), df=np.int64(11))

In [9]:
cnn_label == resnet_label

np.True_